In [2]:
import pandas as pd
import os
import re
import json
import yaml
from glob import glob
from tqdm import tqdm
from pprint import pprint
import torch
import pytorch_lightning as pl
from rouge import Rouge # 모델의 성능을 평가하기 위한 라이브러리입니다.

from torch.utils.data import Dataset , DataLoader
from transformers import AutoTokenizer, BartForConditionalGeneration, BartConfig
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import Trainer, TrainingArguments
from transformers import EarlyStoppingCallback

In [3]:
from googletrans import Translator
import re

In [15]:
def backtranslate(text, src_lang='ko', target_lang='en'):
    text = text.split('\n')
    length = len(text)

    translator = Translator()

    result = ''

    for i in range(length):
        person = text[i][:11]
        dia = text[i][11:]
        try:
            translate_text = translator.translate(dia, src=src_lang, dest=target_lang).text
            back_translate_text = translator.translate(translate_text, src=target_lang, dest=src_lang).text
        except Exception as e:
            print(f"An error occurred during translation: {e}")
            return -1
        test_result = person + back_translate_text  
        
        if i != length - 1:
            test_result = test_result + '\n'

        result = result + test_result

    return result


In [16]:
input_text = "#Person1#: 안녕하세요, 스미스씨. 저는 호킨스 의사입니다. 오늘 왜 오셨나요? #Person2#: 건강검진을 받는 것이 좋을 것 같아서요. #Person1#: 그렇군요, 당신은 5년 동안 건강검진을 받지 않았습니다. 매년 받아야 합니다."

backtranslated_text = backtranslate(input_text)
backtranslated_text

'#Person1#: 안녕하세요, 스미스 씨. 저는 호킨스 의사입니다. 오늘 왜 왔니?#person2#: 건강 검진을받는 것이 좋은 생각이라고 생각합니다.#Person1#: 예, 5에 대한 건강 검진을받지 못했습니다.몇 년. 당신은 매년 그것을 받아야합니다.'

In [18]:
df = pd.read_csv("/root/NLP/data/train.csv")

In [19]:
df

,fname,dialogue,summary,topic
0,train_0,"#Person1#: 안녕하세요, 스미스씨. 저는 호킨스 의사입니다. 오늘 왜 오셨나...","스미스씨가 건강검진을 받고 있고, 호킨스 의사는 매년 건강검진을 받는 것을 권장합니...",건강검진 받기
1,train_1,"#Person1#: 안녕하세요, 파커 부인, 어떻게 지내셨나요?\n#Person2#...",파커 부인이 리키를 데리고 백신 접종을 하러 갔다. 피터스 박사는 기록을 확인한 후...,백신
2,train_2,"#Person1#: 실례합니다, 열쇠 한 묶음 보셨나요?\n#Person2#: 어떤...","#Person1#은 열쇠 한 묶음을 찾고 있고, 그것을 찾기 위해 #Person2#...",열쇠 찾기
3,train_3,#Person1#: 왜 너는 여자친구가 있다는 걸 말해주지 않았어?\n#Person...,#Person1#은 #Person2#가 여자친구가 있고 그녀와 결혼할 것이라는 사실...,여자친구가 있다
4,train_4,"#Person1#: 안녕, 숙녀분들! 오늘 밤 당신들은 정말 멋져 보여. 이 춤을 ...",말릭이 니키에게 춤을 요청한다. 말릭이 발을 밟는 것을 신경 쓰지 않는다면 니키는 ...,댄스
...,...,...,...,...
12452,train_12455,#Person1#: 실례합니다. 맨체스터 출신의 그린 씨이신가요?\n#Person2...,탄 링은 흰머리와 수염으로 쉽게 인식되는 그린 씨를 만나 호텔로 데려갈 예정입니다....,누군가를 태우다
12453,train_12456,#Person1#: 이윙 씨가 우리가 컨퍼런스 센터에 오후 4시에 도착해야 한다고 ...,#Person1#과 #Person2#는 이윙 씨가 늦지 않도록 요청했기 때문에 컨퍼...,컨퍼런스 센터
12454,train_12457,#Person1#: 오늘 어떻게 도와드릴까요?\n#Person2#: 차를 빌리고 싶...,#Person2#는 #Person1#의 도움으로 5일 동안 소형 차를 빌립니다.,차 렌트
12455,train_12458,#Person1#: 오늘 좀 행복해 보이지 않아. 무슨 일 있어?\n#Person2...,#Person2#의 엄마가 일자리를 잃었다. #Person2#는 엄마가 우울해하지 ...,실직


In [ ]:
s_idx = len(df)
except_list = dict()

for i in tqdm(range(len(df))):
    idx = s_idx + i
    df.loc[idx, 'fname'] = f'train_{idx}'
    df.loc[idx, 'topic'] = df.loc[i, 'topic']
    df.loc[idx, 'summary'] = df.loc[i, 'summary']
    dia = backtranslate(text=df.loc[i, 'dialogue'])
    if dia != -1 :
        df.loc[idx, 'dialogue'] = dia 
    else:
        except_list[idx] = i
        print(f'index: {i}에서 google translate api 오류 발생')

  1%|          | 119/12457 [27:59<61:17:55, 17.89s/it]

An error occurred during translation: the JSON object must be str, bytes or bytearray, not NoneType
index: 118에서 google translate api 오류 발생


  5%|▍         | 565/12457 [2:41:35<48:41:38, 14.74s/it] 

An error occurred during translation: the JSON object must be str, bytes or bytearray, not NoneType
index: 564에서 google translate api 오류 발생


  5%|▍         | 566/12457 [2:41:35<34:52:49, 10.56s/it]

An error occurred during translation: the JSON object must be str, bytes or bytearray, not NoneType
index: 565에서 google translate api 오류 발생


  6%|▌         | 768/12457 [3:41:09<75:56:26, 23.39s/it]

An error occurred during translation: the JSON object must be str, bytes or bytearray, not NoneType
index: 767에서 google translate api 오류 발생


  6%|▋         | 806/12457 [3:53:29<49:37:26, 15.33s/it]

In [1]:
df.to_csv("/root/NLP/data/nltk_bt_train.csv", index=False)

NameError: name 'df' is not defined